In [2]:
import time
import datetime
import hf_hydrodata as hf
import xarray as xr
from parflow import read_pfb

def get_snowtel_data(site_id, start_date, num_days):
    """
    Get snowtel average temp data from one site for num_days starting at start_date.
    Returns: (data, x, y) where x,y are conus2 grid location of the site.
    """
    site_filter_options = {"dataset": "snotel", "variable": "air_temp", "site_id": site_id}
    site_path = hf.get_paths(site_filter_options) [0]
    ds = xr.open_dataset(site_path)
    lat = ds.attrs["latitude"]
    lon = ds.attrs["longitude"]
    (x, y) = hf.to_ij("conus2", lat, lon)
    da = ds["temp_avg"]
    end_date = (datetime.datetime.strptime(start_date, "%Y-%m-%d") + datetime.timedelta(days=num_days-1)).strftime("%Y-%m-%d")
    data = da.sel(date=slice(start_date, end_date)).values
    data = data + 273.5
    return (data, x, y)

def get_cw3e_data(x, y, start_date, num_days):
    """
    Get CW3E mean temp data for num_days starting at start_date.
    Return: a numpy array with dimension (time, y, x) with shape (num_days, 1, 1) with temp data.
    """
    result = None
    grid_bounds = [x, y, x + 1, y + 1]
    start_time = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    for day in range(0, num_days):
        end_time = start_time + datetime.timedelta(hours=24)
        cw3e_filter_options = {
            "dataset": "CW3E", "variable": "air_temp", "temporal_resolution": "hourly", 
            "grid_bounds": grid_bounds, "start_time": start_time, "end_time":end_time}
        cw3e_data = hf.get_gridded_data(cw3e_filter_options)
        if result is None:
            result = []
        v = float(cw3e_data.mean(axis=0)[0][0])
        result.append(v)
        start_time = start_time + datetime.timedelta(hours = 24)
    return result

def get_new_cw3e_data(x, y, start_date, num_days):
    """
    Get CW3E mean temp data for num_days starting at start_date.
    Return: a numpy array with dimension (time, y, x) with shape (num_days, 1, 1) with temp data.
    """
    result = []
    grid_bounds = [x, y, x + 1, y + 1]
    start_time = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    for _ in range(0, num_days):
        wy = start_time.year if start_time.month < 10 else start_time.year+1
        wy_start = datetime.datetime.strptime(f"{wy-1}-10-01", "%Y-%m-%d")
        hr_start = int((start_time - wy_start).total_seconds() / 3600) + 1
        hr_end = hr_start + 23
        pfb_path = f"/hydrodata/temp/CW3E_Forcing_Update_21_Jan_24/processing/processed_data/hourly/WY{wy}/CW3E.Temp.{hr_start:06d}_to_{hr_end:06d}.pfb"
        pfb_constraint = {
            "x": {"start": int(x), "stop": int(x)},
            "y": {"start": int(y), "stop": int(y)},
            "z": {"start": 0, "stop": 24},
        }
        data = read_pfb(pfb_path, pfb_constraint)
        v = data.mean()
        result.append(v)
        start_time = start_time + datetime.timedelta(hours = 24)
    return result
 

def print_site_cw3e_differences(site_id, start_date, num_days, tolerance, pfb_version="old"):
    """
    Print the difference in average temperature for the observation site compared with CW3E data from num_days from start_date
    Only print values that diff by more than 1 degree Kelvin.
    """
    (site_data, x, y) = get_snowtel_data(site_id, start_date, num_days)
    cw3e_data = get_cw3e_data(x, y, start_date, num_days)
    new_cw3e_data = get_new_cw3e_data(x, y, start_date, num_days)
    

    num_old_diffs = 0
    num_new_diff = 0
    for i in range(0, num_days):
        old_value = cw3e_data[i]
        new_value = new_cw3e_data[i]
        diff_old = site_data[i] - old_value
        diff_new = site_data[i] - new_value
        if abs(diff_old) > tolerance or abs(diff_new) > tolerance:
            if abs(diff_old) > tolerance:
                num_old_diffs = num_old_diffs + 1
            if abs(diff_new) > tolerance:
                num_new_diffs = num_new_diff + 1
            date = (datetime.datetime.strptime(start_date, "%Y-%m-%d") + datetime.timedelta(days = i)).strftime("%Y-%m-%d")
            print(f"{site_id} {date} Site {site_data[i]}, CW3E {old_value} NEW CW3E {new_value} OLD DIFF {diff_old} NEW_DIFF {diff_new}")
    print(f"#New {num_old_diffs} #Old {num_old_diffs} difference greater than {tolerance}.")

site_ids = ["350:WY:SNTL", "347:MT:SNTL", "368:UT:SNTL", "396:UT:SNTL", "913:CO:SNTL", "2090:AR:SCAN"]
site_ids = ["301:CA:SNTL"]

start_time = "2009-10-01"
num_days = 365
for site_id in site_ids:
    print_site_cw3e_differences(site_id, start_time, num_days, 6 )




301:CA:SNTL 2009-10-27 Site 272.1, CW3E 278.1122512817383 NEW CW3E 273.6813481648763 OLD DIFF -6.0122512817382585 NEW_DIFF -1.5813481648762604
301:CA:SNTL 2009-10-30 Site 280.0, CW3E 272.42510350545246 NEW CW3E 276.6898600260417 OLD DIFF 7.574896494547545 NEW_DIFF 3.3101399739583144
301:CA:SNTL 2009-11-18 Site 269.6, CW3E 277.8383433024089 NEW CW3E 273.83496856689453 OLD DIFF -8.23834330240885 NEW_DIFF -4.2349685668945085
301:CA:SNTL 2009-12-11 Site 271.0, CW3E 263.54689915974933 NEW CW3E 266.63701248168945 OLD DIFF 7.45310084025067 NEW_DIFF 4.362987518310547
301:CA:SNTL 2010-01-01 Site 275.6, CW3E 269.4090220133464 NEW CW3E 273.21706517537433 OLD DIFF 6.19097798665365 NEW_DIFF 2.3829348246256927
301:CA:SNTL 2010-02-11 Site 274.6, CW3E 267.6537742614746 NEW CW3E 271.3529523213704 OLD DIFF 6.946225738525413 NEW_DIFF 3.247047678629599
301:CA:SNTL 2010-03-15 Site 277.6, CW3E 271.12206904093426 NEW CW3E 274.6870651245117 OLD DIFF 6.477930959065759 NEW_DIFF 2.912934875488304
301:CA:SNTL 201